In [4]:
from google.colab import drive
drive.mount('/content/drive') #shift + Enter

Mounted at /content/drive


In [5]:
import os
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, Input, Activation, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint

In [6]:
root_path=r'/content/drive/MyDrive/dataset/fer'
emotions = os.listdir(root_path)

In [7]:
import cv2

In [8]:
happy_items=os.listdir(os.path.join(root_path, 'happy'))
len(happy_items)

1774

In [9]:
emotions

['sad', 'neutral', 'happy']

In [10]:
print(emotions.index('neutral'))

1


In [11]:

from keras.utils import to_categorical
import numpy as np

In [12]:
train_x = []
train_y = []

for emotion in emotions:
  full_path = os.path.join(root_path,emotion)
  imgs = os.listdir(full_path)
  for img in imgs:
    img_path = os.path.join(full_path, img)
    vector_img = image.load_img(img_path, target_size=(224,224))
    vector_img = image.img_to_array(vector_img)
    vector_img = preprocess_input(vector_img)
    train_x.append(vector_img)
    train_y.append(emotions.index(emotion))

train_x = np.asarray(train_x)
train_y = to_categorical(train_y)

In [13]:
train_x.shape, train_y.shape

((4254, 224, 224, 3), (4254, 3))

In [14]:
from keras.models import Model
from sklearn.model_selection import train_test_split

In [15]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=7)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=7)

In [16]:
train_x.shape, val_x.shape, test_x.shape, train_y.shape, val_y.shape, test_y.shape

((2722, 224, 224, 3),
 (681, 224, 224, 3),
 (851, 224, 224, 3),
 (2722, 3),
 (681, 3),
 (851, 3))

In [24]:
base_model=ResNet50(input_shape=(224,224,3))
#base_model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable=False
x=base_model.get_layer('avg_pool').output
x=Dense(3)(x)
for layer in base_model.layers[-26:]:
    layer.trainable = True

model_save_path=r'/content/drive/MyDrive/saved_models/model_weights.h5'
model_checkpoint_callback=ModelCheckpoint(model_save_path, monitor='val_loss', mode='min', save_weights_only=True, save_best_only=True)
model = Model(inputs=base_model.inputs, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=10, batch_size=32, callbacks=[model_checkpoint_callback])

In [ ]:
#model.load_weights(model_save_path)
test_loss, test_acc = model.evaluate(test_x, test_y)

print('test acc:', test_acc)